In [1]:
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from pandas_datareader import data as web
import matplotlib.dates as mdates
%matplotlib inline
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
#make plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
init_notebook_mode(connected=True)
#Use Plotly locally
cf.go_offline()
import warnings
warnings.simplefilter('ignore')
import yfinance as yf
from plotly.subplots import make_subplots
#install the python Technical Analysis
#python -m pip install ta
from ta.trend import MACD
from ta.momentum import StochasticOscillator

In [3]:
#Valid period: 1d,5d,1mo,3mo,6mo,1y,2y,5y10y,,ytd,max
#valid intervals: 1m,2m,5m,15m,30m,60m,90m,,1h,1d,5d,1wk,1mo,3mo
amd_df =yf.download(tickers='AMD',period="1y",interval='1d')
amd_df

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-03-15,81.910004,83.389999,81.199997,82.500000,82.500000,39086800
2021-03-16,83.660004,84.750000,82.260002,82.750000,82.750000,42971600
2021-03-17,81.750000,83.250000,80.410004,82.629997,82.629997,43320600
2021-03-18,81.059998,81.620003,78.010002,78.120003,78.120003,51036500
2021-03-19,78.489998,79.339996,77.589996,79.059998,79.059998,42283400
...,...,...,...,...,...,...
2022-03-08,102.809998,109.900002,100.080002,105.529999,105.529999,135348300
2022-03-09,108.410004,111.709999,106.849998,111.050003,111.050003,102310300
2022-03-10,108.889999,109.070000,103.070000,106.459999,106.459999,102557400


In [3]:
#Exponential Moving Average
amd_df['MA20'] = amd_df['Adj Close'].ewm(span=20,adjust=False).mean()
amd_df['MA50'] = amd_df['Adj Close'].ewm(span=50,adjust=False).mean()
amd_df

,Open,High,Low,Close,Adj Close,Volume,MA20,MA50
Date,,,,,,,,
2021-03-12,79.730003,81.190002,79.230003,81.050003,81.050003,32938000,81.050003,81.050003
2021-03-15,81.910004,83.389999,81.199997,82.500000,82.500000,39086800,81.188098,81.106866
2021-03-16,83.660004,84.750000,82.260002,82.750000,82.750000,42971600,81.336851,81.171302
2021-03-17,81.750000,83.250000,80.410004,82.629997,82.629997,43320600,81.460007,81.228506
2021-03-18,81.059998,81.620003,78.010002,78.120003,78.120003,51036500,81.141912,81.106604
...,...,...,...,...,...,...,...,...
2022-03-07,108.529999,109.529999,102.820000,102.949997,102.949997,92600000,115.831341,122.079345
2022-03-08,102.809998,109.900002,100.080002,105.529999,105.529999,135348300,114.850261,121.430351
2022-03-09,108.410004,111.709999,106.849998,111.050003,111.050003,102310300,114.488331,121.023278


In [4]:
#Plotting Moving Average
fig = go.Figure()
candle = go.Candlestick(x=amd_df.index, open=amd_df['Open'],
                        high=amd_df['High'],
                        low=amd_df['Low'],
                        close=amd_df['Close'],
                        name= 'Candlestic')
ema20= go.Scatter(x=amd_df.index,y=amd_df['MA20'],
                 line=dict(color='green',width=1),name='EMA20')
ema50= go.Scatter(x=amd_df.index,y=amd_df['MA50'],
                 line=dict(color='purple',width=1),name='EMA50')
fig.add_trace(candle)
fig.add_trace(ema20)
fig.add_trace(ema50)
fig.update_layout(title='AMD')
fig.update_xaxes(rangebreaks=[dict(bounds=["sat","mon"])])
fig.show()

In [5]:
def plot_macd_stoch(ticker, time_period, time_int):
    # Download stock data
    stock_df = yf.download(tickers=ticker, period=time_period, interval=time_int)
    
    # Define candlestick and moving average lines
    candle = go.Candlestick(x=stock_df.index, open=stock_df['Open'],
                       high=stock_df['High'], low=stock_df['Low'],
                       close=stock_df['Close'])
    # A period of 12 and 26 are commonly used
    stock_df['MA12'] = stock_df['Adj Close'].ewm(span=12, adjust=False).mean()
    stock_df['MA26'] = stock_df['Adj Close'].ewm(span=26, adjust=False).mean()
    
    # Calculate the MACD
    # A signal line uses a period of 9 and when it crosses the other moving
    # average it is a sign to buy or sell
    macd = MACD(close=stock_df['Close'],
             window_slow=26,
             window_fast=12,
             window_sign=9)

    # A Stochastic (stuh ka stuhk) Oscillator is also plotted because it 
    # gives us a signal of an upcoming trend reversal
    # Values range from 0 to 100 and values over 80 are considered to be
    # overbought while those under 20 are considered oversold
    
    # We calculate normally over a 14 day period
    # We are smoothing price data
    sto_os = StochasticOscillator(high=stock_df['High'],
                             close=stock_df['Close'],
                             low=stock_df['Low'],
                             window=14, 
                             smooth_window=3)
    
    # Make 4 subplots on separate rows
    fig = make_subplots(rows=4, cols=1, shared_xaxes=True, vertical_spacing=0.01)

    fig.add_trace(candle, row=1, col=1)

    # Add the histogram, MACD and Signal lines
    
    # The MacD line is a calculation made by subtracting the 26 period
    # EMA from the 12 period EMA
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd(),
                         line=dict(color='blue', width=2)
                        ), row=2, col=1)
    
    # The signal is calculated by taking the average of the last 9 values
    # of the MACD line (The signal line is a slower more smoothed out version of
    # the MACD line)
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd_signal(),
                         line=dict(color='orange', width=2)
                        ), row=2, col=1)
    
    # The histogram shows the difference between the MACD and signal line
    # When the MACD line is below the signal that is a negative value and vice versa
    fig.add_trace(go.Bar(x=stock_df.index, 
                     y=macd.macd_diff()
                    ), row=2, col=1)
    
    # The MacD line is a calculation made by subtracting the 26 period
    # EMA from the 12 period EMA
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd(),
                         line=dict(color='blue', width=2)
                        ), row=2, col=1)
    
    # The signal is calculated by taking the average of the last 9 values
    # of the MACD line (The signal line is a slower more smoothed out version of
    # the MACD line)
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=macd.macd_signal(),
                         line=dict(color='orange', width=2)
                        ), row=2, col=1)

    # Plot Stochastics
    # This is the faster of the 2 lines called the K line
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=sto_os.stoch(),
                         line=dict(color='blue', width=2)
                        ), row=3, col=1)
    # This line is slower and is known as the D line and it is an average of the K line
    fig.add_trace(go.Scatter(x=stock_df.index,
                         y=sto_os.stoch_signal(),
                         line=dict(color='orange', width=2)
                        ), row=3, col=1)
    
    # Draw 20 and 80 lines
    fig.add_hline(y=20, line_width=1, line_dash="dash", line_color="red", row=3, col=1)
    fig.add_hline(y=80, line_width=1, line_dash="dash", line_color="green", row=3, col=1)

    # Add volume 
    fig.add_trace(go.Bar(x=stock_df.index, 
                     y=stock_df['Volume']
                    ), row=4, col=1)

    # Update titles
    fig.update_layout(title=ticker)
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="MACD", row=2, col=1)
    fig.update_yaxes(title_text="StoOs", row=3, col=1)
    fig.update_yaxes(title_text="Volume", row=4, col=1)

    fig.update_layout(height=900, width=1200, 
                  showlegend=False, 
                  xaxis_rangeslider_visible=False,
                  xaxis_rangebreaks=[dict(bounds=["sat", "mon"]), dict(bounds=[16, 9.5], pattern="hour")])

    fig.show()

In [6]:
plot_macd_stoch("PG","5D","5m")

[*********************100%***********************]  1 of 1 completed
